installing the required libraries

In [3]:
!pip install -q transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00


loading the dataset

In [2]:
from google.colab import files
uploaded = files.upload()


Saving claims.csv to claims.csv


In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv('claims.csv')

# adding claim id col
df.insert(0, "ClaimID", range(1, len(df) + 1))

df.head()

,ClaimID,ReportingDelay,AccidentHour,Age,WeeklyRate,Gender,MaritalStatus,HoursWorkedPerWeek,DependentChildren,DependentsOther,PartTimeFullTime,DaysWorkedPerWeek,DateOfAccident,ClaimDescription,ReportedDay,InitialCaseEstimate,Incurred
0,1,15,11,28,500.00,M,S,44.0,0,0,F,5,5/10/2005,STRUCK SCAFFOLDING STRAIN LOWER BACK,3Thurs,9500,151.254501
1,2,22,5,29,500.00,M,S,38.0,0,0,F,5,28/10/2003,STRUCK KNIFE LACERATED LEFT THUMB,2Wed,11500,442.125024
2,3,22,7,28,197.37,M,M,16.0,0,0,P,3,25/05/2004,SLIPPED AND HIT STRAINED LEFT SHOULDER INJURY ...,2Wed,8000,1494.490505
3,4,15,12,40,0.00,M,M,0.0,1,0,F,5,21/07/1994,HIT FALLING DOOR LACERATION LEFT SHOULDER INJU...,4Fri,500,138.900000
4,5,38,12,22,435.70,M,M,38.0,0,0,F,5,9/06/1992,STRUCK FALLING OBJECT LACERATION RIGHT RING FI...,4Fri,320,296.160000


loading the model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "tiiuae/falcon-7b-instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in 8-bit (memory efficient) mode
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # fallback to float16 if Colab GPU doesn't support bf16
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

testing the model tiiuae/falcon-7b-instruct



In [7]:
claim = "STRUCK SCAFFOLDING STRAIN LOWER BACK"

prompt = f"Convert the following keywords into a complete sentence:\n\nKeywords: {claim.lower().capitalize()}\nSentence:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Convert the following keywords into a complete sentence:

Keywords: Struck scaffolding strain lower back
Sentence: The worker's lower back was strained when scaffolding collapsed.


testing the model on dataset

In [10]:
# Create a copy of the first 10 rows to avoid modifying the whole DataFrame yet
subset_df = df.head(10).copy()

# Function to generate a full sentence from ClaimDescription
def generate_detailed_description(claim):
    prompt = (
        f"Convert the following injury keywords into a realistic claim statement. "
        f"The sentence should sound like something a person would say when reporting the incident.\n\n"
        f"Keywords: {claim.lower().capitalize()}\n"
        f"Statement:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_output.replace(prompt, "").strip()

# Apply to first 10 rows
subset_df["DetailedClaimDescription"] = subset_df["ClaimDescription"].apply(generate_detailed_description)

# Show result
subset_df[["ClaimDescription", "DetailedClaimDescription"]]


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


,ClaimDescription,DetailedClaimDescription
0,STRUCK SCAFFOLDING STRAIN LOWER BACK,I suffered a lower back strain from a falling ...
1,STRUCK KNIFE LACERATED LEFT THUMB,I suffered a laceration to my left thumb when ...
2,SLIPPED AND HIT STRAINED LEFT SHOULDER INJURY ...,I slipped and hit my left shoulder while using...
3,HIT FALLING DOOR LACERATION LEFT SHOULDER INJU...,"I suffered an injury, including a hit by a fal..."
4,STRUCK FALLING OBJECT LACERATION RIGHT RING FI...,I was injured when a falling object struck my ...
5,KNIFE SLIPPED CUT LEFT INDEX FINGER,I accidentally cut my left index finger with a...
6,SLIPPED ON STEEL BEAM LOWER BACK BRUISING KNEE...,"I slipped on a steel beam, resulting in lower ..."
7,LIFTING METAL SHEETS STRAIN LOWER BACK PAIN,I sustained a lower back injury while attempti...
8,LIFTING PATIENT BACK PAIN,I sustained back pain while lifting the patien...
9,SLIPPED OFF TRUCK BRUISED LOWER BACK RIGHT KNEE,"I slipped off a truck, resulting in bruises on..."


In [ ]:
import os
import pandas as pd

# Load the original dataset if not already loaded
# Uncomment and modify the file path if needed
# df = pd.read_csv("your_dataset.csv")

# Output file path
output_file = "claims_with_detailed_descriptions.csv"

# Set ClaimID to resume from
resume_claim_id = 3301
start_index = resume_claim_id - 1  # Convert ClaimID to 0-based index

# Load previous results if any
if os.path.exists(output_file):
    result_df = pd.read_csv(output_file)
    print(f"Existing output found. Will resume from ClaimID {resume_claim_id}.")
else:
    result_df = pd.DataFrame()
    print("No existing output file. Starting fresh.")

# Batch processing settings
batch_size = 150
total_rows = len(df)

# Loop through the dataset starting from the desired index
for start in range(start_index, total_rows, batch_size):
    end = min(start + batch_size, total_rows)
    print(f"Processing rows {start + 1} to {end}")

    batch = df.iloc[start:end].copy()
    batch["DetailedClaimDescription"] = batch["ClaimDescription"].apply(generate_detailed_description)

    # Append or write to CSV
    if os.path.exists(output_file):
        batch.to_csv(output_file, mode='a', header=False, index=False)
    else:
        batch.to_csv(output_file, index=False)

    print(f"Saved rows {start + 1} to {end} to file.")


applying function to dataset in batches

In [ ]:
import os
import pandas as pd

# Load original dataset again if needed
# df = pd.read_csv("your_dataset.csv")

# File to save results incrementally
output_file = "claims_with_detailed_descriptions.csv"

# Determine starting point
if os.path.exists(output_file):
    result_df = pd.read_csv(output_file)
    start_id = result_df["ClaimID"].max() + 1
    print(f"Resuming from ClaimID {start_id}")
else:
    result_df = pd.DataFrame()
    start_id = 1
    print("Starting from scratch")

batch_size = 150
total_rows = len(df)

for start in range(start_id - 1, total_rows, batch_size):
    end = min(start + batch_size, total_rows)
    print(f"Processing rows {start + 1} to {end}")

    batch = df.iloc[start:end].copy()
    batch["DetailedClaimDescription"] = batch["ClaimDescription"].apply(generate_detailed_description)

    # Append to result file
    if os.path.exists(output_file):
        batch.to_csv(output_file, mode='a', header=False, index=False)
    else:
        batch.to_csv(output_file, index=False)

    print(f"Saved batch {start + 1}–{end} to file.")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Starting from scratch
Processing rows 1 to 150


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Saved batch 1–150 to file.
Processing rows 151 to 300


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

...

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Saved batch 3151–3300 to file.
Processing rows 3301 to 3450


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Saved batch 3301–3450 to file.
Processing rows 3451 to 3600


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok